In [76]:
import sqlite3

# Connect to DB
connexion = sqlite3.connect("../data/db/vivino.db")
cursor = connexion.cursor()

In [55]:
# cursor.execute("""
#    SELECT
        
#    FROM
    
#    WHERE
               
#    GROUP BY
               
#    HAVING
               
#    ORDER BY
        
#    LIMIT ;
#""")
# cursor.fetchall()


## 2. We have a marketing budget. Which countries should we prioritise, and why?

- countries
  - name
  - regions_count
  - users_count
  - wines_count
  - wineries_count
  - population

In [56]:
# Number of users by country top 10

top_10_filter = """
SELECT
    name AS country_name,
    users_count AS number_of_users,
    population AS population
FROM
    countries
ORDER BY
    countries.users_count DESC
LIMIT
    10;
"""

cursor.execute(top_10_filter)
top_10_filter = cursor.fetchall()
display(top_10_filter)


[('États-Unis', 12273684, 339996564),
 ('France', 5973301, 64756584),
 ('Italie', 4270717, 58870763),
 ('Allemagne', 2549989, 83294633),
 ('Espagne', 2264396, 47519628),
 ('Suisse', 1601799, 8796669),
 ('Portugal', 1123535, 10247605),
 ('Australie', 1022965, 26439112),
 ('Argentine', 629532, 45773884),
 ('Chili', 326757, 19629590)]

In [57]:
# Proportion of populatuon as users top_10
population_proportion = """
SELECT
    countries.name,
    round((CAST(countries.users_count AS FLOAT)/CAST(countries.population AS FLOAT)*100),2)
FROM
    countries
ORDER BY
    round(CAST(countries.users_count AS FLOAT)/CAST(countries.population AS FLOAT),2) DESC
LIMIT
    10;
"""

cursor.execute(population_proportion)
population_proportion = cursor.fetchall()

print('Percentage of population using Vivino:\n')

for i in population_proportion:
    print(str(i[0])+': '+str(i[1])+'%')

Percentage of population using Vivino:

Suisse: 18.21%
Portugal: 10.96%
France: 9.22%
Italie: 7.25%
Espagne: 4.77%
États-Unis: 3.61%
Australie: 3.87%
Allemagne: 3.06%
Chili: 1.66%
Israël: 1.64%


In [58]:
# Number of users per winery in country
wineries_per_country = """
SELECT
    countries.name,
    CAST(ROUND(CAST(countries.users_count AS FLOAT)/CAST(countries.wineries_count AS FLOAT)*100) AS INTEGER) AS users_per_winery_percent
FROM
    countries
GROUP BY
    countries.name
ORDER BY
    users_per_winery_percent 
"""

cursor.execute(wineries_per_country)
wineries_per_country = cursor.fetchall()

print('Number of users per winery per country:\n')

for i in wineries_per_country:
    print(str(i[0]+': '+ str(i[1])))

Number of users per winery per country:

Moldavie: 3250
Hongrie: 5316
Chili: 5648
Afrique du Sud: 6379
Croatie: 6553
Australie: 7335
Grèce: 7395
France: 8842
Italie: 10073
Argentine: 11791
Espagne: 12562
Allemagne: 18691
Portugal: 19258
Israël: 28459
Roumanie: 33263
Suisse: 41616
États-Unis: 43609


# 2. Average wine rating for each country. Do the same for the vintages.

In [59]:
# Average wine rating for each country

wine_per_country = """
SELECT
    countries.name,
    round(avg(wines.ratings_average),3) AS wine_rating_avg,
    sum(wines.ratings_count)
FROM
    countries
INNER JOIN regions 
    ON countries.code = regions.country_code
INNER JOIN wines
    ON regions.id = wines.region_id
GROUP BY 
    countries.name
ORDER BY
    wine_rating_avg DESC;
"""
cursor.execute(wine_per_country)
wine_per_country = cursor.fetchall()

print('Average wine rating per country (3 d.p.):\n')
for i in wine_per_country:
    print(str(i[0])+': '+ str(i[1])+' calulcated from '+str(i[2])+' ratings.')

Average wine rating per country (3 d.p.):

Israël: 4.5 calulcated from 894 ratings.
Allemagne: 4.5 calulcated from 8257 ratings.
États-Unis: 4.491 calulcated from 834263 ratings.
Moldavie: 4.48 calulcated from 14114 ratings.
Hongrie: 4.473 calulcated from 30564 ratings.
Afrique du Sud: 4.459 calulcated from 59995 ratings.
Australie: 4.458 calulcated from 52138 ratings.
France: 4.447 calulcated from 2124809 ratings.
Espagne: 4.444 calulcated from 746710 ratings.
Portugal: 4.436 calulcated from 62454 ratings.
Chili: 4.431 calulcated from 159256 ratings.
Italie: 4.43 calulcated from 2135839 ratings.
Argentine: 4.417 calulcated from 283673 ratings.
Roumanie: 4.4 calulcated from 166 ratings.
Grèce: 4.4 calulcated from 610 ratings.
Suisse: 4.35 calulcated from 1047 ratings.
Croatie: 4.3 calulcated from 292 ratings.


In [60]:
vintage_per_country = """
SELECT
    countries.name,
    round(avg(vintages.ratings_average),3) AS vintage_rating_avg,
    sum(vintages.ratings_count)
FROM
    countries
INNER JOIN regions 
    ON countries.code = regions.country_code
INNER JOIN wines
    ON regions.id = wines.region_id
INNER JOIN vintages
    ON wines.id = vintages.wine_id
WHERE
    vintages.ratings_average >= 4.4
GROUP BY 
    countries.name
ORDER BY
    vintage_rating_avg DESC;
"""
cursor.execute(vintage_per_country)
vintage_per_country = cursor.fetchall()

print('Average vintage rating per country (3 d.p.):\n')
for i in vintage_per_country:
    print(str(i[0])+': '+ str(i[1])+' calulcated from '+str(i[2])+' ratings.')

Average vintage rating per country (3 d.p.):

Grèce: 4.6 calulcated from 62 ratings.
France: 4.556 calulcated from 402781 ratings.
États-Unis: 4.552 calulcated from 102030 ratings.
Australie: 4.53 calulcated from 3723 ratings.
Italie: 4.523 calulcated from 465884 ratings.
Allemagne: 4.522 calulcated from 780 ratings.
Espagne: 4.519 calulcated from 48689 ratings.
Moldavie: 4.517 calulcated from 4172 ratings.
Portugal: 4.515 calulcated from 5487 ratings.
Roumanie: 4.5 calulcated from 31 ratings.
Afrique du Sud: 4.5 calulcated from 5558 ratings.
Argentine: 4.481 calulcated from 16930 ratings.
Chili: 4.478 calulcated from 14066 ratings.
Hongrie: 4.44 calulcated from 2981 ratings.
Suisse: 4.4 calulcated from 162 ratings.
Croatie: 4.4 calulcated from 234 ratings.


# Nice to have feature 3 - Cabernet Sauvignon

In [61]:
cabernet_sauvignon_vintage = """
SELECT
    vintages.name,
    vintage_toplists_rankings.rank,
    vintages.ratings_average,
    vintages.ratings_count,
    vintages.price_euros
FROM
    vintages
INNER JOIN
    vintage_toplists_rankings
    ON vintages.id = vintage_toplists_rankings.vintage_id
WHERE
    vintages.name LIKE '%Cabernet Sauvignon%'

"""
cursor.execute(cabernet_sauvignon_vintage)
cabernet_sauvignon_vintage = cursor.fetchall()

cabernet_sauvignon_vintage

[('Scarecrow Cabernet Sauvignon 2015', 1, 4.8, 243, 1558.75),
 ('Buena Vista Chateau Buena Vista Cabernet Sauvignon Napa Valley 2018',
  1,
  4.4,
  3867,
  61.95)]

In [71]:
# Find the statistics for Scarecrow 2015 vintage

scarecrow_stats = """
SELECT
    wines.ratings_average AS rating,
    wines.acidity AS acidity,
    wines.fizziness AS fizziness,
    wines.intensity AS intensity,
    wines.sweetness AS sweetness,
    wines.tannin AS tannin
FROM
    wines 
INNER JOIN vintages
    ON wines.id = vintages.wine_id  
WHERE
    vintages.name = 'Scarecrow Cabernet Sauvignon 2015';
"""

cursor.execute(scarecrow_stats)
scarecrow_stats = cursor.fetchall()

for row in scarecrow_stats:
    print(row)


(4.8, 3.304091, None, 4.7008004, 1.7583516, 3.3939934)


In [81]:
# Find a list of 5 wines that are similar to Scarecrow 2015 vintage using the above ratings

similar_to_scarecrow = """
SELECT
    vintages.name,
    vintages.price_euros,
    vintages.ratings_average
FROM 
    vintages
INNER JOIN wines 
    ON vintages.wine_id  = wines.id
WHERE
    (wines.acidity BETWEEN 3.2 AND 3.4)
    AND wines.fizziness IS NULL
    AND (wines.intensity BETWEEN 4.6 AND 4.8)
    AND (wines.sweetness BETWEEN 1.65 AND 1.85)
    AND (wines.tannin BETWEEN 3.3 AND 3.5)
"""
cursor.execute(similar_to_scarecrow)
similar_to_scarecrow = cursor.fetchall()

similar_to_scarecrow

[('Scarecrow Cabernet Sauvignon 2015', 1558.75, 4.8),
 ('Duckhorn Rutherford Cabernet Sauvignon 2017', 147.5, 4.6),
 ('Joseph Phelps Vineyards Cabernet Sauvignon 2018', 85.95, 4.4),
 ('Joseph Phelps Vineyards Cabernet Sauvignon 2019', 95.95, 4.5),
 ('Hourglass Blueline Estate Cabernet Sauvignon 2019', 195.0, 4.5)]